In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Подготовка данных
### Импорт данных:

In [2]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

### Функция подготовки данных по признакам HouseFloor и Floor:
1. если этаж квартиры больше этажности дома, то значения этаж-этажность меняютс местами
2. если этажность дома больше порогового значения threshold (по умолчанию 50), то этажность принимается равной этажу квартиры
3. нулевая этажность дома приравнивается к этажу дома
4. нулевые этажи приравниваются первому этажу

In [3]:
def prepare_floor_house_floor(df, threshold=50):
    _ind = df.loc[df['Floor']>df['HouseFloor'],:].index
    _var = df.loc[df['Floor']>df['HouseFloor'],['HouseFloor','Floor']]
    df.loc[_ind,'Floor'] = _var['HouseFloor']
    df.loc[_ind,'HouseFloor'] = _var['Floor']
    df.loc[df['HouseFloor']>threshold,'HouseFloor'] = df.loc[df['HouseFloor']>threshold,'Floor']
    df.loc[df['HouseFloor']==0,'HouseFloor'] = df.loc[df['HouseFloor']==0,'Floor']
    df.loc[df['Floor']==0,'Floor'] = 1
    return df

#### подготовка train.csv:
* правка выбросов по HouseYear

In [4]:
data.loc[1497,'HouseYear'] = 2011
data.loc[4189,'HouseYear'] = 1968
data = pd.get_dummies(data)
data = prepare_floor_house_floor(data)

#### подготовка test.csv:
* в HouseYear выбросов нет

In [5]:
test_data = pd.get_dummies(test_data)
test_data = prepare_floor_house_floor(test_data)

### Функции подготовки в данных по различным признакам:
prepare_rooms:

* sq_per_room - среднее отношение жилой площади к количеству комнат (квартиры с 0 комнат в расчет не берутся)
* квартиры, имеющие количество комнат больше max_rooms расчитываются как отношение жилой площади к sq_per_room
* пропущенные значения расчитываются аналогично, но вместо жилой площади учитывается общая

In [6]:
def prepare_rooms(df, max_rooms=5):
    sq_per_room = (df.loc[df['Rooms']!=0,'LifeSquare']/df.loc[df['Rooms']!=0,'Rooms']).mean()    
    df.loc[df['Rooms'] > max_rooms, 'Rooms'] = round((df.loc[df['Rooms'] > max_rooms, 'LifeSquare'])/sq_per_room)
    df.loc[pd.isna(df['Rooms']), 'Rooms'] = round((df.loc[pd.isna(df['Rooms']), 'Square'])/sq_per_room)
    return df

prepare_square:

* все квартиры, имеющие площадь меньше порогового значения threshold (160 м2) и количество комнат 1, 2 или 3, группируются по количеству комнат (3 группы).
* _med_sq - среднее значение площади квартиры по группам
* все квартиры с количеством комныт 1, 2 или 3 превышающие по площади пороговое значение принимают среднее значение площади _med_sq в зависимости от количества комнат

In [7]:
def prepare_square(df, threshold=160):
    _true_data = df.loc[(df['Square'] < threshold) & (df['Rooms'] > 0) & (df['Rooms'] < 4)]
    _med_sq = _true_data.groupby(['Rooms'])[['Square']].mean()      #MEAN!!!!!!!!!!!!!!!!!!!!!!!!!!
    for i in (1,2,3):
        df.loc[(df['Square'] > threshold) & (df['Rooms'] == i),'Square'] = _med_sq.loc[i,'Square']
    return df

prepare_healthcare_1:

* df_hc_1 медианные значения Healthcare_1 сгрупированные по районам
* отсутствующие в df_hc_1 значения заполняются средним по df_hc_1
* отсутствующие значения признака Healthcare_1 заменяются средним по текущему району из df_hc_1

In [8]:
def prepare_healthcare_1(df):
    df_hc_1 = df.groupby(['DistrictId'])[['Healthcare_1']].median()
    df_hc_1['Healthcare_1'] = df_hc_1['Healthcare_1'].fillna(df_hc_1['Healthcare_1'].mean())
    for d_ind in df_hc_1.index:
        df.loc[(pd.isna(df['Healthcare_1'])) & (df['DistrictId'] == d_ind),'Healthcare_1'] = df_hc_1.loc[d_ind,'Healthcare_1']
    return df

prepare_kitchen:

* если площадь кухни больше max_sq, ее значение заменяется медианным по всей выборке

In [9]:
def prepare_kitchen(df, max_sq = 25):
    df.loc[df['KitchenSquare']>max_sq,'KitchenSquare'] = df['KitchenSquare'].median()
    return df

### Функции создания в данных новых признаков:
add_mean_price_per_square - дебаляет среднюю цену за квадратный метр в районе

In [10]:
def add_mean_price_per_square(df_X_train, df_y_train, df_valid, on_features=['DistrictId']):
    df_train = df_X_train.join(df_y_train)
    df_train['price_per_square'] = df_train['Price']/df_train['Square']
    stat = df_train.groupby(on_features,as_index=False)[['price_per_square']].mean().rename(columns={'price_per_square':'mean_price_per_sq'})
    df_train = df_train.merge(stat, on=on_features,how='left').drop('price_per_square',axis=1)
    df_valid = df_valid.merge(stat, on=on_features,how='left')
    _meanval = df_valid['mean_price_per_sq'].mean()
    df_valid['mean_price_per_sq'] = df_valid['mean_price_per_sq'].fillna(_meanval)
    return df_train.drop('Price',axis=1), df_valid

add_mean_price - добавляет среднюю цену, в зависимости от количества комнат

In [11]:
def add_mean_price(df_X_train, df_y_train, df_valid, on_features=['Rooms']):
    df_train = df_X_train.join(df_y_train)
    stat = df_train.groupby(on_features,as_index=False)[['Price']].mean().rename(columns={'Price':'mean_price'})
    df_train = df_train.merge(stat, on=on_features,how='left')
    _meantrain = df_train['Price'].mean()
    df_train['mean_price'] = df_train['mean_price'].fillna(_meantrain)
    df_valid = df_valid.merge(stat, on=on_features,how='left')
    _meanvalid = df_valid['mean_price'].mean()
    df_valid['mean_price'] = df_valid['mean_price'].fillna(_meanvalid)
    return df_train.drop('Price',axis=1), df_valid

add_mean_square - добавляет среднюю площадь в районе и в зависимости от количества комант

In [12]:
def add_mean_square(df, on_features=['DistrictId','Rooms']):
    stat = df.groupby(on_features,as_index=False)[['Square']].mean().rename(columns={'Square':'mean_Square'})
    return df.merge(stat, on=on_features,how='left')

### Общая функция подготовки данных:

In [13]:
def preproc_data(x_train_, y_train_, x_test):
    x_train_, x_test = prepare_kitchen(x_train_), prepare_kitchen(x_test)
    x_train_, x_test = prepare_rooms(x_train_), prepare_rooms(x_test)
    x_train_, x_test = prepare_square(x_train_), prepare_square(x_test)
    x_train_, x_test = prepare_healthcare_1(x_train_), prepare_healthcare_1(x_test)
    
    x_train_, x_test = add_mean_price_per_square(x_train_, y_train_, x_test)
    x_train_, x_test = add_mean_square(x_train_), add_mean_square(x_test)
    x_train_, x_test = add_mean_price(x_train_, y_train_, x_test)
        
    x_train_, x_test = x_train_.drop(['LifeSquare','Id'],axis=1), x_test.drop(['LifeSquare','Id'],axis=1)
    return x_train_, x_test

# Валидация модели
### Разделение на тренировочную и валидационную выборки на базе данных train.csv

In [14]:
from sklearn.model_selection import train_test_split
X = data.drop('Price', axis=1)
y = pd.DataFrame(data['Price'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=1)

### Подготовка данных для обучения и валидации:

In [15]:
X_train, X_valid = preproc_data(X_train, y_train, X_valid)

### Применение модели RandomForestRegressor
#### Создание и обучение модели, предсказание на тестовой и валидационной выборках:

In [16]:
from sklearn.ensemble import RandomForestRegressor

model_rfr = RandomForestRegressor(random_state=1, n_estimators=150, max_depth=12, max_features = 5)
model_rfr.fit(X_train, y_train.values[:, 0])
y_pred_rfr = model_rfr.predict(X_valid)
y_pred_train_rfr = model_rfr.predict(X_train)

#### Оценка R2 на тестовой и валидационной выборках:

In [17]:
from sklearn.metrics import r2_score

r2val = r2_score(y_valid, y_pred_rfr)
r2tra = r2_score(y_train, y_pred_train_rfr)

print('# r2_valid: ', r2val)  
print('# r2_train: ', r2tra)
print('# diff: ', r2tra - r2val)

# r2_valid:  0.7369359003887752
# r2_train:  0.9053216193206162
# diff:  0.16838571893184096


#### Анализ наиболее важных признаков:

In [18]:
def get_feature_importances(model, feature_names, count=None):
    _data = {'feature_name' : feature_names, 'feature_importances' : model.feature_importances_}
    return pd.DataFrame(data=_data).sort_values('feature_importances', ascending=False).head(count)

begin, end = 0, 8
print(f'SUM_importances from {begin} to {end}: ', get_feature_importances(model_rfr, np.array(X_train.columns))[begin:end].feature_importances.values.sum())
get_feature_importances(model_rfr, np.array(X_train.columns))[begin:end]

SUM_importances from 0 to 8:  0.777442380354143


,feature_name,feature_importances
2,Square,0.204616
20,mean_price_per_sq,0.159463
21,mean_Square,0.105253
1,Rooms,0.097641
22,mean_price,0.072515
9,Social_2,0.056827
3,KitchenSquare,0.041063
8,Social_1,0.040065


# Предсказание цены на данных test.csv

### Разделение на итоговую тренировочную и тестовую выборки

In [19]:
X_main_train = data.drop('Price', axis=1)
y_main_train = pd.DataFrame(data['Price'])
X_main_test = test_data

### Подготовка данных:

In [20]:
X_main_train, X_main_test = preproc_data(X_main_train, y_main_train, X_main_test)

### Применение модели RandomForestRegressor
#### Создание и обучение модели на тренировочной выборке, предсказание на тестовой выборке:

In [21]:
from sklearn.ensemble import RandomForestRegressor

fin_model = RandomForestRegressor(random_state=1, n_estimators=150, max_depth=12, max_features = 5)
fin_model.fit(X_main_train, y_main_train.values[:, 0])
test_data['Price'] = fin_model.predict(X_main_test)
y_train_pred  = fin_model.predict(X_main_train)

#### Оценка R2 на тренировочной выборке:

In [22]:
from sklearn.metrics import r2_score

r2train = r2_score(y_main_train, y_train_pred)

print('# r2_train: ', r2train)

# r2_train:  0.8956360938009785


#### Анализ наиболее важных признаков:

In [23]:
def get_feature_importances(model, feature_names, count=None):
    _data = {'feature_name' : feature_names, 'feature_importances' : model.feature_importances_}
    return pd.DataFrame(data=_data).sort_values('feature_importances', ascending=False).head(count)

begin, end = 0, 5
print(f'SUM_importances from {begin} to {end}: ', get_feature_importances(fin_model, np.array(X_main_train.columns))[begin:end].feature_importances.values.sum())
get_feature_importances(fin_model, np.array(X_main_train.columns))[begin:end]

SUM_importances from 0 to 5:  0.6512962648731999


,feature_name,feature_importances
2,Square,0.229108
20,mean_price_per_sq,0.165556
21,mean_Square,0.087845
1,Rooms,0.087586
22,mean_price,0.081202


### Cохранение результата:

In [24]:
test_data[['Id','Price']].to_csv('predictions.csv',index=None)

In [25]:
test_data[['Id','Price']].head(5)

,Id,Price
0,725,161388.287716
1,15856,227194.364745
2,5480,275827.462682
3,15664,298332.307642
4,14275,137377.234191
